# MDL DB read

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['lines.linewidth'] = 1.5

import os
import copy

import data
from mdldb.mdl_db import MDLDataBase
from mdldb.tables import Run
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.analytical_linear_estimator import AnalyticalLinearEstimator
from rolldecayestimators.direct_linear_estimator import DirectLinearEstimator
from rolldecayestimators.direct_estimator_cubic import DirectEstimatorCubic
from rolldecayestimators.direct_estimator import DirectEstimator
from rolldecayestimators.norwegian_estimator import NorwegianEstimator



from mdldb import mdl_to_evaluation
from evaluation.run_dynamic import RunDynamic
from evaluation.run_manoeuvring import RunZigZag
from sklearn.pipeline import Pipeline
import signal_lab

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///' + data.mdl_db_path)
db = MDLDataBase(engine=engine)

In [ ]:
db.session.query(Run.test_type).distinct().all()

In [ ]:
len(db.session.query(Run).filter(Run.test_type=='roll decay').all())

In [ ]:
roll_decay_tests = db.session.query(Run).filter(Run.test_type=='roll decay')

In [ ]:
db_run = roll_decay_tests[100]
#db_run = roll_decay_tests[150]

In [ ]:
ascii_file = db_run.load()
df_raw = ascii_file.channels

df = signal_lab.mdl_to_evaluation.do_transforms(df=df_raw)
df.rename(columns={'MA/Roll':'phi'}, inplace=True)

In [ ]:
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=2, minimum_score=0.99)
scaler = ScaleFactorTransformer(scale_factor=db_run.model.scale_factor)  # dummy value None for now
cutter = CutTransformer(phi_max=np.deg2rad(7), phi_min=np.deg2rad(1))

steps = [('filter',lowpass_filter),
         ('scaler',scaler),
         ('cutter', cutter),
        ]

preprocess = Pipeline(steps)
X = preprocess.fit_transform(df)

In [ ]:
X.plot(y='phi')

In [ ]:
estimators = []
estimators.append(DirectLinearEstimator(omega_regression=True))
estimators.append(AnalyticalLinearEstimator(omega_regression=True))
estimators.append(DirectEstimator(omega_regression=True))
estimators.append(NorwegianEstimator())
estimators.append(DirectEstimatorCubic(omega_regression=True))


for estimator in estimators:

    estimator.fit(X)
    
    fig,ax=plt.subplots()
    fig.set_size_inches(14,10)
    estimator.plot_fit(ax=ax)
    score = estimator.score()
    
    title = ''
    if estimator.omega_regression:
        title+='Omega regression '
    title+='Score:%0.2f' % score 
    ax.set_title(title)